# Spark DataFrame Basics

## Load File into DataFrame

In [122]:
from pyspark.sql import SparkSession

In [123]:
spark = SparkSession.builder.appName("Basics").getOrCreate()

In [124]:
df = spark.read.json('people.txt')

## Display DataFrame contents

`dataframe.show()`

In [125]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



### Describe (summarize) DataFrame contents

`DataFrame.describe()` summarizes and returns a new DataFrame

In [126]:
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



### Get Column(s)

In [127]:
df.columns

['age', 'name']

In [128]:
df['age']

Column<'age'>

In [129]:
df[0]

Column<'age'>

### Get Row(s)

In [130]:
df.head(2)

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

In [131]:
df.head(1)[0]

Row(age=None, name='Michael')

### Selecting Columns

Selecting a column `df.select('age')`, or columns `df.select(['age', 'name])` returns a DataFrame

In [132]:
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [133]:
df.select(['age', 'name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



## DataFrame Schema

### Display the DataFrame Schema

In [134]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [135]:
type(df.schema)

pyspark.sql.types.StructType

## Create a Schema

In [136]:
from pyspark.sql.types import (
    StructField,
    StringType,
    IntegerType,
    StructType
)

### Create a Typed Column

`StructField('age', IntegerType(), True)`

where true means Nulls allowed

In [137]:
data_schema = [
    StructField('age', IntegerType(), True),
    StructField('name', StringType(), True),
]
final_schema = StructType(fields=data_schema)

In [138]:
data_schema = [
    StructField('age', IntegerType(), True),
    StructField('name', StringType(), True),
]
final_schema = StructType(fields=data_schema)

### Load Data using Self Defined Schema

In [139]:
custom_df = spark.read.json('people.txt', schema=final_schema)

In [140]:
custom_df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



## Modifying DataFrames

### Creating New Columns

In [141]:
age_col = custom_df['age']
custom_df.withColumn('new_age', age_col).show()

+----+-------+-------+
| age|   name|new_age|
+----+-------+-------+
|null|Michael|   null|
|  30|   Andy|     30|
|  19| Justin|     19|
+----+-------+-------+



In [142]:
custom_df.withColumn('double_age', age_col*2).show()

+----+-------+----------+
| age|   name|double_age|
+----+-------+----------+
|null|Michael|      null|
|  30|   Andy|        60|
|  19| Justin|        38|
+----+-------+----------+



### Rename a Column

In [143]:
custom_df.withColumnRenamed('age', 'new_age').show()

+-------+-------+
|new_age|   name|
+-------+-------+
|   null|Michael|
|     30|   Andy|
|     19| Justin|
+-------+-------+



## SQL Operations On a DataFrame

### Use sql to interact with DataFrame

1. Create a Temp TABLE from DataFrame
2. Run sql against TABLE
3. Results are a new DataFrame

In [144]:
custom_df.createOrReplaceTempView('people')

In [145]:
results = spark.sql("SELECT * FROM PEOPLE")

In [146]:
results.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [147]:
spark.sql("SELECT * FROM PEOPLE WHERE AGE > 20").show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



In [148]:
type(spark.sql("SELECT * FROM PEOPLE"))

pyspark.sql.dataframe.DataFrame